In [2]:
import numpy as np
import pandas as pd
import io

In [3]:
df = pd.read_csv("2 Machine Learning Dataset\\2 DataScientistJobs.csv")

# 看Data Information ，把info函數轉成我們好看的dataframe

In [ ]:
def get_data_description(data: pd.DataFrame):
    buffer = io.StringIO()
    data.info(buf=buffer)
    lines = buffer.getvalue().splitlines()
    details = (pd.DataFrame([x.split() for x in lines[5:-2]], columns=lines[3].split())
               .drop('Count', axis=1)
               .rename(columns={'Non-Null': 'Non-Null Count'})
               .rename(columns={'#': 'number'}))
    unique_value = []
    for i in data.columns:
        unique_value.append(len(data[i].unique()))
    details['unique'] = unique_value
    return details

因為在上面的函數中，我使用了split()，沒有特別調，但是在這次檔案中columns中是有空白的，因此會出錯，我將columns有空白的部分加了底線， 讓我能使用上面的函數。

In [6]:
x = [x.replace(" ", "_") for x in df.columns]
df.columns = x
df_final = df.copy()
df_final = df_final.iloc[:, 2:]  #去除index和沒用的column
df_final

,Job_Title,Salary_Estimate,Job_Description,Rating,Company_Name,Location,Headquarters,Size,Founded,Type_of_ownership,Industry,Sector,Revenue,Competitors,Easy_Apply
0,Senior Data Scientist,$111K-$181K (Glassdoor est.),"ABOUT HOPPER\n\nAt Hopper, we’re on a mission ...",3.5,Hopper\n3.5,"New York, NY","Montreal, Canada",501 to 1000 employees,2007,Company - Private,Travel Agencies,Travel & Tourism,Unknown / Non-Applicable,-1,-1
1,"Data Scientist, Product Analytics",$111K-$181K (Glassdoor est.),"At Noom, we use scientifically proven methods ...",4.5,Noom US\n4.5,"New York, NY","New York, NY",1001 to 5000 employees,2008,Company - Private,"Health, Beauty, & Fitness",Consumer Services,Unknown / Non-Applicable,-1,-1
2,Data Science Manager,$111K-$181K (Glassdoor est.),Decode_M\n\nhttps://www.decode-m.com/\n\nData ...,-1.0,Decode_M,"New York, NY","New York, NY",1 to 50 employees,-1,Unknown,-1,-1,Unknown / Non-Applicable,-1,True
3,Data Analyst,$111K-$181K (Glassdoor est.),Sapphire Digital seeks a dynamic and driven mi...,3.4,Sapphire Digital\n3.4,"Lyndhurst, NJ","Lyndhurst, NJ",201 to 500 employees,2019,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,"Zocdoc, Healthgrades",-1
4,"Director, Data Science",$111K-$181K (Glassdoor est.),"Director, Data Science - (200537)\nDescription...",3.4,United Entertainment Group\n3.4,"New York, NY","New York, NY",51 to 200 employees,2007,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"BBDO, Grey Group, Droga5",-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3904,AWS Data Engineer,$55K-$112K (Glassdoor est.),About Us\n\nTachyon Technologies is a Digital ...,4.4,Tachyon Technologies\n4.4,"Dublin, OH","Irving, TX",201 to 500 employees,2011,Company - Private,IT Services,Information Technology,$10 to $25 million (USD),-1,-1
3905,Data Analyst â Junior,$55K-$112K (Glassdoor est.),"Job description\nInterpret data, analyze resul...",5.0,"Staffigo Technical Services, LLC\n5.0","Columbus, OH","Woodridge, IL",51 to 200 employees,2008,Company - Private,IT Services,Information Technology,$50 to $100 million (USD),-1,-1
3906,Security Analytics Data Engineer,$55K-$112K (Glassdoor est.),Job DescriptionThe Security Analytics Data Eng...,3.8,"PDS Tech, Inc.\n3.8","Dublin, OH","Irving, TX",5001 to 10000 employees,1977,Company - Private,Staffing & Outsourcing,Business Services,$100 to $500 million (USD),-1,-1
3907,Security Analytics Data Engineer,$55K-$112K (Glassdoor est.),The Security Analytics Data Engineer will inte...,4.0,Data Resource Technologies\n4.0,"Dublin, OH","Omaha, NE",1 to 50 employees,-1,Company - Private,Accounting,Accounting & Legal,Less than $1 million (USD),-1,-1


In [7]:
df_final.describe()

,Rating,Founded
count,3909.000000,3909.000000
mean,3.283576,1479.154771
std,1.575749,855.750138
min,-1.000000,-1.000000
25%,3.200000,1625.000000
50%,3.700000,1976.000000
75%,4.100000,2002.000000
max,5.000000,2020.000000


In [8]:
df_detail = get_data_description(df)
df_detail

,number,Column,Non-Null Count,Dtype,unique
0,0,Unnamed:_0,3909,int64,3909
1,1,index,3909,int64,3909
2,2,Job_Title,3909,object,2079
3,3,Salary_Estimate,3909,object,148
4,4,Job_Description,3909,object,3685
5,5,Rating,3909,float64,37
6,6,Company_Name,3909,object,2075
7,7,Location,3909,object,191
8,8,Headquarters,3909,object,543
9,9,Size,3909,object,9


# Simple Feature Engineering
要做EDA之前，要先針對比較重要的變數做特別的處理。

### 薪資間距處理(Salary Estimate)

Salary Estimate要特別去處理一種是，Per Hour的薪資 因此我把它統一弄成年薪的樣子，並假設一年是52week，一周工作40hours

In [10]:
def hour_salary_deal(st):
    """
    :param st:Salary裡的字串
    :type st:
    :return: 處理後的字串
    :rtype: str
    """
    hours_week = 40
    weeks_year = 52
    if " Per Hour" in st:
        st_final=st.replace("(Glassdoor est.)","").replace("(Employer est.)","")
        st_final = st_final.replace(" Per Hour", "")
        st_up = int(st_final.split("-")[0].replace('$', '')) * hours_week * weeks_year
        st_down = int(st_final.split("-")[1].replace('$', ''))* hours_week * weeks_year

    else:
        st_final=st.replace("(Glassdoor est.)","").replace("(Employer est.)","")
        st_up = int(
            st_final.split("-")[0].replace("$", "").replace("K", "000")
        )
        st_down = int(
            st_final.split("-")[1].replace("$", "").replace("K", "000")
        )
    return [st_up,st_down]


df_final["Salary_cond"]=df_final['Salary_Estimate'].apply(lambda st:hour_salary_deal(st))
df_final[['Salary_cond','Salary_Estimate']]

,Salary_cond,Salary_Estimate
0,"[111000, 181000]",$111K-$181K (Glassdoor est.)
1,"[111000, 181000]",$111K-$181K (Glassdoor est.)
2,"[111000, 181000]",$111K-$181K (Glassdoor est.)
3,"[111000, 181000]",$111K-$181K (Glassdoor est.)
4,"[111000, 181000]",$111K-$181K (Glassdoor est.)
...,...,...
3904,"[55000, 112000]",$55K-$112K (Glassdoor est.)
3905,"[55000, 112000]",$55K-$112K (Glassdoor est.)
3906,"[55000, 112000]",$55K-$112K (Glassdoor est.)
3907,"[55000, 112000]",$55K-$112K (Glassdoor est.)
